In [15]:
import tensorflow as tf
from time import time
import numpy
import os
import json
import pickle
import pandas
import datetime
from functools import partial, reduce
import importlib

import sys
sys.path.append('../libs')

import data_pipeline
import conv_model
import initialize
import prepare_data
import flacdb
import plot_batch
import loss_metrics
import generate_report

In [2]:
! nvidia-smi

Wed Nov 20 22:47:22 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.78       Driver Version: 410.78       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX TIT...  On   | 00000000:89:00.0 Off |                  N/A |
| 22%   42C    P2    68W / 250W |      0MiB / 12212MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [8]:
! ls /scr1/checkpoints/ | grep 1131251

1131251_20191120-141715_00128.ckpt.data-00000-of-00002
1131251_20191120-141715_00128.ckpt.data-00001-of-00002
1131251_20191120-141715_00128.ckpt.index
1131251_20191120-141715_00256.ckpt.data-00000-of-00002
1131251_20191120-141715_00256.ckpt.data-00001-of-00002
1131251_20191120-141715_00256.ckpt.index
1131251_20191120-141715_00384.ckpt.data-00000-of-00002
1131251_20191120-141715_00384.ckpt.data-00001-of-00002
1131251_20191120-141715_00384.ckpt.index
1131251_20191120-141715_00384.ckpt.pkl
1131251_20191120-141715_00512.ckpt.data-00000-of-00002
1131251_20191120-141715_00512.ckpt.data-00001-of-00002
1131251_20191120-141715_00512.ckpt.index
1131251_20191120-141715_00640.ckpt.data-00000-of-00002
1131251_20191120-141715_00640.ckpt.data-00001-of-00002
1131251_20191120-141715_00640.ckpt.index
1131251_20191120-141715_00768.ckpt.data-00000-of-00002
1131251_20191120-141715_00768.ckpt.data-00001-of-00002
1131251_20191120-141715_00768.ckpt.index
1131251_20191120-141715_00768.ckpt.pkl
1131251_20191120

In [90]:
ls /scr1/checkpoints/ | grep pkl

1131088_20191120-074645_00768.ckpt.pkl
1131088_20191120-074645_00896.ckpt.pkl
1131088_20191120-074645_01024.ckpt.pkl
1131251_20191120-141715_00384.ckpt.pkl


In [9]:
model_id = '1131251_20191120-141715'
checkpoint = str(896).zfill(5)
checkpoint_path = '/scr1/checkpoints/{}_{}.ckpt'.format(model_id, checkpoint)
H0 = json.load(open('../hypes/{}.json'.format(model_id)))

In [3]:
%%time

H = initialize.load_hypes()
K = ['layer_count_a', 'layer_count_b', 'kernel_sizes', 'filter_count']
K += ['dense_units', 'strides', 'window_size']
for k in K:
    H[k + '_log2'] = H0[k + '_log2']
    
initial_data_path = '/scr1/mimic/initial_data/'

# sig_data, metadata = initialize.load_initial_data(save_path=initial_data_path)
sig_data, metadata = initialize.load_initial_data(load_path=initial_data_path)
diagnosis = initialize.load_diagnosis(H['icd_codes'], metadata)
diagnosis = initialize.augment_diagnosis(diagnosis, metadata)
diagnosis = initialize.fix_diagnosis(diagnosis)
priors = (diagnosis == 1).sum() / (diagnosis != 0).sum()
priors['measured_systemic_hypertension'] = 0.5
priors['measured_pulmonary_hypertension'] = 0.5
diagnosis = initialize.conform_diagnosis(diagnosis, metadata)
partition = initialize.load_partition(H['input_sigs_validation'], sig_data)

CPU times: user 3.12 s, sys: 272 ms, total: 3.39 s
Wall time: 3.39 s


In [4]:
%%time

I = partition['validation']
row_lengths = initialize.get_row_lengths(metadata[I])
args = [metadata[I], sig_data[I], diagnosis[I], row_lengths]
tensors = initialize.get_tensors(H, *args)
H = {**H, 'batch_size_validation_log2': 7}
dataset = data_pipeline.build(H, tensors, 'validation')

CPU times: user 3.65 s, sys: 644 ms, total: 4.3 s
Wall time: 4.28 s


In [10]:
%%time

# model = conv_model.build(H, priors)
model.load_weights(checkpoint_path)

CPU times: user 824 ms, sys: 360 ms, total: 1.18 s
Wall time: 987 ms


In [11]:
%%time

batch_count = 2**8
fix_input = lambda x: {**x, 'mask': tf.cast(x['mask'], 'float')}
if os.path.isfile(checkpoint_path + '.pkl'):
    with open(checkpoint_path + '.pkl', 'rb') as f:
        X, Y, P = pickle.load(f)
else:
    X, Y, P = generate_report.compute_predictions(
        model, dataset, batch_count, fix_input)
    with open(checkpoint_path + '.pkl', 'wb') as f:
        pickle.dump([X, Y, P], f)

CPU times: user 12min 15s, sys: 17.8 s, total: 12min 33s
Wall time: 7min 15s


In [78]:
# importlib.reload(loss_metrics)
importlib.reload(generate_report)

<module 'generate_report' from '../libs/generate_report.py'>

In [16]:
%matplotlib widget
codes = diagnosis.columns.to_list()
codes += ['measured_systemic_hypertension', 'measured_pulmonary_hypertension']
y_true, y_pred = Y['diagnosis'], P['diagnosis']
x = X['signals'][:, :, [H['input_sigs'].index(i) for i in ['PLETH', 'II']]]
M = metadata.reset_index()[['subject_id', 'rec_id']].drop_duplicates()
M = M.set_index('rec_id', verify_integrity=True)
subject_ids = M.loc[Y['rec_id']].values[:, 0]
fig, plotter = generate_report.get_diagnoses_plotter(
    x, y_true, y_pred, codes, subject_ids)
plotter();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(SelectionSlider(continuous_update=False, description='Condition:', layout=Layout(width='…

In [17]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
mask (InputLayer)               [(None, 11)]         0                                            
__________________________________________________________________________________________________
signals (InputLayer)            [(None, 512, 11)]    0                                            
__________________________________________________________________________________________________
tf_op_layer_ExpandDims (TensorF [(None, 1, 11)]      0           mask[0][0]                       
__________________________________________________________________________________________________
tf_op_layer_ExpandDims_2 (Tenso [(None, 512, 11, 1)] 0           signals[0][0]                    
______________________________________________________________________________________________